In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from tensorflow import keras
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_global_policy(policy)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model created")


27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
49 unique lemmatized words ["'d", "'s", 'a', 'accept', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'later', 'like', 'mastercard', 'moped', 'of', 'only', 'open', 'rent', 'see', 'take', 'thank', 'thanks', 'that', 'there', 'this', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'you', 'your']
Training data created


C:\Users\anuj\AppData\Local\Temp\ipykernel_15968\1655658899.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
6/6 [==============================] - 1s 5ms/step - loss: 2.3131 - accuracy: 0.1481
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 2.1319 - accuracy: 0.0370
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 2.2156 - accuracy: 0.1852
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1027 - accuracy: 0.2222
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0061 - accuracy: 0.2222
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0041 - accuracy: 0.1481
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8533 - accuracy: 0.4074
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7821 - accuracy: 0.4444
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6965 - accuracy: 0.5556
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7167 - accuracy: 0.3333
Epoch 11/200
6/6 [===========

6/6 [==============================] - 0s 3ms/step - loss: 0.1164 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0429 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1885 - accuracy: 0.9259
Epoch 87/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0940 - accuracy: 0.9630
Epoch 88/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1007 - accuracy: 0.9630
Epoch 89/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0566 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1064 - accuracy: 1.0000
Epoch 91/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2476 - accuracy: 0.9259
Epoch 92/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1730 - accuracy: 0.9630
Epoch 93/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2508 - accuracy: 0.9259
Epoch 94/200
6/6 [===============

6/6 [==============================] - 0s 3ms/step - loss: 0.1215 - accuracy: 0.9630
Epoch 168/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0577 - accuracy: 0.9630
Epoch 169/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0571 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0451 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0838 - accuracy: 0.9630
Epoch 173/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0350 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0536 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0492 - accuracy: 0.9630
Epoch 176/200
6/6 [==============================] - 0s 3ms/step - loss: 0.2123 - accuracy: 0.9259
Epoch 177/200
6/6 [=====